In [8]:
import os
import networkx as nx
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
import pandas as pd

OUTPUT_PATH = "./resources/output/"

In [9]:
# Shapefile 데이터 로드
file_path = "resources/IMCRTS_Mini/imcrts_link_mini.shp"  # 여기에 실제 파일 경로를 입력하세요.
imcrts_link_mini: gpd.GeoDataFrame = gpd.read_file(file_path)

# 데이터 확인
sample = imcrts_link_mini.head()
print(sample)
sample.to_excel(os.path.join(OUTPUT_PATH, "imcrts_link_mini_sample.xlsx"))

# TURNINFO 데이터 로드
turninfo_path = "resources/IMCRTS_Mini/TURNINFO.dbf"  # 여기에 실제 파일 경로를 입력하세요.
turninfo = gpd.read_file(turninfo_path)

# 데이터 확인
sample = turninfo.head()
print(sample)
sample.to_excel(os.path.join(OUTPUT_PATH, "TURNINFO_sample.xlsx"))

      LINK_ID      F_NODE      T_NODE  LANES ROAD_RANK ROAD_TYPE ROAD_NO  \
0  1640147500  1640047800  1640049000      4       104       000       -   
1  1640144300  1640046600  1640047800      3       104       000       -   
2  1640144500  1640047800  1640046600      3       104       000       -   
3  1640110700  1640037500  1640036200      3       104       000       -   
4  1640105200  1640035600  1640034600      3       104       000       -   

     ROAD_NAME ROAD_USE MULTI_LINK  ... REST_VEH  REST_W REST_H  C-ITS  \
0  �۵���������        0          0  ...        0       0      0      0   
1   ��ũ����ũ��        0          0  ...        0       0      0      0   
2   ��ũ����ũ��        0          0  ...        0       0      0      0   
3   ��õŸ������        0          0  ...        0       0      0      0   
4   ��õŸ������        0          0  ...        0       0      0      0   

       LENGTH UPDATEDATE  REMARK HIST_TYPE HISTREMARK  \
0  567.223492   20230519    None      Non

In [10]:
# 샘플 데이터로 그래프 생성을 시뮬레이션
def create_graph_from_data(links, turninfo):
    G = nx.Graph()

    # LINK_ID를 노드로 추가
    for link_id in links["LINK_ID"]:
        G.add_node(link_id)

    # 같은 NODE를 공유하는 LINK_ID 사이에 엣지 추가
    for _, row in links.iterrows():
        f_node = row["F_NODE"]
        t_node = row["T_NODE"]

        # F_NODE와 T_NODE를 공유하는 다른 LINK_ID 찾기
        connected_links = links[
            (links["F_NODE"] == f_node) | (links["T_NODE"] == t_node)
        ]
        for _, connected_row in connected_links.iterrows():
            if connected_row["LINK_ID"] != row["LINK_ID"]:
                # 동일 도로상의 서로 다른 방향 LINK 확인
                if not are_opposite_links(row["LINK_ID"], connected_row["LINK_ID"]):
                    G.add_edge(row["LINK_ID"], connected_row["LINK_ID"])

    # TURNINFO 데이터를 사용하여 추가 엣지 추가
    for _, row in turninfo.iterrows():
        if row["TURN_TYPE"] == 11:  # 예시: 유턴 가능인 경우
            G.add_edge(row["ST_LINK"], row["ED_LINK"])

    return G


# 동일 도로상의 서로 다른 방향 LINK인지 확인
def are_opposite_links(link1, link2):
    # LINK_ID의 오른쪽에서 세 번째 숫자 추출
    third_last_digit1 = int(str(link1)[-3])
    third_last_digit2 = int(str(link2)[-3])

    # 서로 다른 방향 LINK 확인
    opposite_pairs = [(1, 2), (3, 4), (5, 6), (7, 8), (9, 0)]
    return any(
        [
            set([third_last_digit1, third_last_digit2]) == set(pair)
            for pair in opposite_pairs
        ]
    )


# 그래프 생성
G1 = create_graph_from_data(imcrts_link_mini, turninfo)
G1

In [11]:
# CSV 파일 읽기
distance_imc_path = './resources/IMCRTS_Micro_Dataset_Train/sensor_graph/distance_imc.csv'  # 실제 파일 경로로 교체해야 함
distance_imc_df = pd.read_csv(distance_imc_path)

# 그래프 생성
G2 = nx.from_pandas_edgelist(distance_imc_df, 'from', 'to', ['cost'])


In [12]:
import networkx as nx
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv


# Function to load and preprocess data
def load_data(file_path: str):
    # Load data (modify as per your file type)
    data = pd.read_hdf(file_path)
    # Normalize and preprocess your data here
    return data


# Function to build graph from sensor data
def build_graph(sensor_file: str):
    sensor_data: pd.DataFrame = pd.read_csv(sensor_file)
    G = nx.from_pandas_edgelist(sensor_data, "from", "to")
    edge_index = torch.tensor(list(G.edges)).t().contiguous()
    return edge_index


# PyTorch Geometric GCN model
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


# Load and preprocess data
train_data = load_data("./resources/IMCRTS_Micro_Dataset_Train/imcrts_df.h5")
test_data = load_data("./resources/IMCRTS_Micro_Dataset_Test/imcrts_df.h5")

# Building the sensor graph
edge_index = build_graph(
    "./resources/IMCRTS_Micro_Dataset_Test/sensor_graph/distance_imc.csv"
)

# Preparing the data for PyTorch Geometric
# Assuming train_data and test_data are in a suitable tensor format
# If not, convert them to tensor and normalize/scale if necessary
train_dataset = Data(x=train_data, edge_index=edge_index)
test_dataset = Data(x=test_data, edge_index=edge_index)

# Model parameters
num_features = train_data.shape[1]
num_classes = 1  # Assuming a regression problem

# Instantiate the model
model = GCN(num_features, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [14]:
train_dataset

Data(x=                     1610001700  1610001800  1610001900  1610002000  \
2023-10-01 00:00:00         9.0        17.0         0.0         0.0   
2023-10-01 01:00:00        14.0        25.0         6.0         0.0   
2023-10-01 02:00:00         0.0         3.0         8.0         0.0   
2023-10-01 03:00:00         3.0        11.0         0.0         0.0   
2023-10-01 04:00:00         4.0         4.0         6.0         8.0   
...                         ...         ...         ...         ...   
2023-11-15 19:00:00       122.0        99.0        76.0        98.0   
2023-11-15 20:00:00       119.0        68.0        54.0        57.0   
2023-11-15 21:00:00        78.0        61.0        29.0        54.0   
2023-11-15 22:00:00        26.0        30.0         0.0        25.0   
2023-11-15 23:00:00        13.0        26.0         6.0        13.0   

                     1610002100  1610002200  1610002302  1610002304  \
2023-10-01 00:00:00         0.0         NaN         0.0         0.0  

In [13]:
# Train the model
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(train_dataset)
    loss = F.mse_loss(out, train_dataset.y)
    loss.backward()
    optimizer.step()

# Prediction and evaluation
model.eval()
pred = model(test_dataset)
# Evaluate your predictions here, e.g., using MSE or MAE

TypeError: 'int' object is not callable